In [1]:
from src.model import get_model

e:\Desktop\apple_sugar_detection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
hidden_dim = 512
config = {
    'spectral_encoder' : {
        'type' : 'resnet1d',
        'layers' : (2, 2),
        'output_dim' : hidden_dim,
        },
    'image_encoder' : {
        'type' : 'multiview',
        'base_encoder' : {
            'type' : 'vit',
            'output_dim' : hidden_dim,
        },
        'num_views' : 2,
        'output_dim' : hidden_dim,
    }
}

model = get_model(
    encoder_config=config,
    output_dim = 1,
    dropout = 0.3,
    output_activation=None
)

In [5]:
batch = 8
spec_c = 1
spec_l = 120

img_v = 2
img_c = 3
img_w = 224
img_h = 224

spectral = torch.zeros((batch, spec_c, spec_l))
imgs = torch.zeros((batch, img_v, img_c, img_w, img_h))
model(spectral = spectral, images = imgs)

spectral_feat:  torch.Size([8, 512])
image_feat:  torch.Size([8, 512])
运行到这了


tensor([0.0453, 0.1420, 0.1827, 0.2347, 0.4028, 0.2129, 0.0915, 0.6816],
       grad_fn=<SqueezeBackward1>)

In [ ]:
model.image_encoder(imgs)

tensor([[ 0.5157, -0.7324, -0.3037,  ...,  0.9009,  0.0932,  0.6033],
        [ 0.5157, -0.7324, -0.3037,  ...,  0.9009,  0.0932,  0.6033],
        [ 0.5157, -0.7324, -0.3037,  ...,  0.9009,  0.0932,  0.6033],
        ...,
        [ 0.5157, -0.7324, -0.3037,  ...,  0.9009,  0.0932,  0.6033],
        [ 0.5157, -0.7324, -0.3037,  ...,  0.9009,  0.0932,  0.6033],
        [ 0.5157, -0.7324, -0.3037,  ...,  0.9009,  0.0932,  0.6033]],
       grad_fn=<AddmmBackward0>)

In [7]:
import pandas as pd

In [9]:
data = pd.read_excel('data/spec.xlsx')

In [11]:
from torch import functional as F

In [ ]:
from torch import nn, optim

from src.model import get_model
from src.dataset import get_data_loaders
from src.trainer import TrainingConfig, Trainer, ModelCheckpointCallback, EarlyStoppingCallback, TensorBoardCallback


# 模型相关
spectral_encoder_type = 'transformer' # 'resnet1d', 'transformer'
image_encoder_type = 'multiview' # 'resnet', 'vit', 'multiview'
output_dim = 1 # 模型输出维度
fusion_method = 'cross_attention' # 'concat', 'bilinear', 'cross_attention'
hidden_dim = 512
dropout = 0.3
output_activation = None
spectral_params = {
     # resnet1d 参数
    'resnet1d' : {
        'layers' : (2, 2),
        # in_channels 共用参数
        # output_dim 共用参数
        # pool_type 共用参数
        'zero_init_residual' : False
    },
    
    # Transformers 参数
    'transformer': {
        'input_channels' : 1,
        'seq_len' : 256,
        'embed_dim' : 64,
        'output_dim' : None,
        'n_heads' : 4,             
        'n_layers' : 3, 
        'expansion_ratio': 4, 
        'dropout': 0.1,
        'use_cnn_preproc': True,
        'pool_type' : 'mean'
    }
}
image_params = {
    'resnet' : {
        'model_name' : 'resnet18',
        'pretrained' : True,
        'freeze_layers' : False,
        'output_dim' : None,
        'pool_type' : 'avg'
    },
    
    'vit' : {
        'model_name' : 'vit_tiny_patch16_224',
        'pretrained' : True,
        'freeze_layers' : False,
        'output_dim' : None,
    }
}
multiview_params = {
    'num_views' : 2,
    'fusion_method' : 'lstm',
    'output_dim' : None,
    'dropout' : 0.1
}

# 数据集相关
img_dir = r'data/fig'
excel_path = r'data/spec.xlsx'
batch_size = 256
img_size = 224
seed = 42
views = 2
spec_preprocess = ['snv']

# 训练相关


model = get_model(
    spectral_encoder_type = 'resnet1d',
    image_encoder_type = 'multiview',
    fusion_method = 'bilinear',
    hidden_dim = 512,
    dropout = dropout,
    output_activation = output_activation,
    spectral_params = spectral_params,
    image_params = image_params,
    multiview_params=multiview_params
)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-08-19 18:26:56,800 - timm.models._builder - INFO - Loading pretrained weights from Hugging Face hub (timm/vit_tiny_patch16_224.augreg_in21k_ft_in1k)
2025-08-19 18:26:57,154 - timm.models._hub - INFO - [timm/vit_tiny_patch16_224.augreg_in21k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


/opt/conda/lib/python3.11/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


In [3]:
import torch

AssertionError: query should be unbatched 2D or batched 3D tensor but received 4-D query tensor

In [ ]:
model.spectral_encoder()